Imports




In [ ]:
%pip install pandas
%pip install tensorflow_decision_forests

In [1]:
import os
import pandas as pd

import tensorflow as tf
import tensorflow_decision_forests as tfdf

## Data Preprocessing

In [40]:
titanic_training = pd.read_csv('titanic_data/train.csv')
titanic_test = pd.read_csv('titanic_data/test.csv')

training_df = titanic_training
testing_df = titanic_test

def preprocess(df, dropId = False):
  df = df.copy()

  def ticket_number(x):
    return x.split(' ')[-1]
  def ticket_item(x):
    items = x.split(" ")
    if len(items) == 1:
      return "NONE"
    return "_".join(items[0:-1])
  if dropId:
    df = df.drop('PassengerId', axis=1)
  df["Ticket_Number"] = df["Ticket"].apply(ticket_number)
  df["Ticket_Item"] = df["Ticket"].apply(ticket_item)
  df["Family_Size"] = df["Parch"] + df["SibSp"] + 1
  df = df.drop(['Name', 'Ticket','Embarked', "Parch", "SibSp"], axis=1)
  df['Age'] = df['Age'].interpolate()
  return df

training_set = preprocess(titanic_training, True)
testing_set = preprocess(titanic_test)

input_features = list(training_set.columns)
input_features.remove('Survived')

input_features

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7a260d518280>
Traceback (most recent call last):
  File "/usr/lib/python3.10/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


['Pclass',
 'Sex',
 'Age',
 'Fare',
 'Cabin',
 'Ticket_Number',
 'Ticket_Item',
 'Family_Size']

## Constructing the model

In [41]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(training_set, label="Survived")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(testing_set)

model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0,
    features = [tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True,
    random_seed=42
)

model.fit(train_ds)

In [42]:
self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")


Accuracy: 0.7961165308952332 Loss:0.9129098653793335


In [44]:
model.summary()

Model: "gradient_boosted_trees_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (8):
	Age
	Cabin
	Family_Size
	Fare
	Pclass
	Sex
	Ticket_Item
	Ticket_Number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  1.000000 ################
    2.        "Pclass"  0.333768 ##
    3.           "Age"  0.250457 #
    4.          "Fare"  0.224448 
    5.   "Ticket_Item"  0.200449 
    6.   "Family_Size"  0.194066 
    7. "Ticket_Number"  0.193805 

Variable Importance: NUM_AS_ROOT:
    1. "Sex" 22.000000 

Variable Importance: NUM_NODES:
    1.           "Age" 183.000000 ################
    2.          "Fare" 121.000000 #########
    3.

## Make Predictions

In [43]:
def make_prediction(model, threshold = 0.5):
  survive_p = model.predict(test_ds, verbose=0)[:,0]
  return pd.DataFrame({
      "PassengerId": testing_df["PassengerId"],
      "Survived": (survive_p > threshold).astype(int)
  })

def make_to_csv(prediction):
  path="titanic_data/prediction.csv"
  prediction.to_csv(path, index=False)

pred = make_prediction(model)
make_to_csv(pred)